In [1]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import pandas as pd

In [37]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [2]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '183aea69820e4dfda0176db816fc1f72' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

In [3]:
# Image path
urlImage2 = 'https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame1000.jpg'
urlImage3= 'https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame1925.jpg'
###s3

In [26]:
url_list = []
for i in range(355,5920,5):
    url = "https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%d.jpg" % i
    url_list.append(url)

'https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame360.jpg'

In [ ]:
#############

In [27]:
# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage} 
data = None
params = None

In [28]:
# Face detect funciton

def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

In [39]:
# Running functions
result = {}
face = {}
for single_image in url_list:
    urlImage = single_image
    result = processRequest( json, data, headers, params,  )

Error code: 400
Message: JSON parsing error.
Error code: 400
Message: JSON parsing error.
Error code: 400
Message: JSON parsing error.
Error code: 400
Message: JSON parsing error.
Error code: 400
Message: JSON parsing error.


KeyboardInterrupt: 

time: 5.04 s


In [13]:
# Adding gender to emotions
final_data = []
for i, value in enumerate(result):
    final_data.append(result[i])
    final_data[i].update(face[i])
final_data

[{'faceAttributes': {'gender': 'male'},
  'faceId': '772a659d-fd44-450a-b075-2a4b1516f072',
  'faceRectangle': {'height': 192, 'left': 275, 'top': 147, 'width': 192},
  'scores': {'anger': 0.251259267,
   'contempt': 0.00229549455,
   'disgust': 0.0468936265,
   'fear': 0.0010336308,
   'happiness': 0.293725938,
   'neutral': 0.223219767,
   'sadness': 0.029654434,
   'surprise': 0.151917815}},
 {'faceAttributes': {'gender': 'female'},
  'faceId': 'fce89e39-f777-4ad9-b0cc-f86a2d139668',
  'faceRectangle': {'height': 176, 'left': 807, 'top': 132, 'width': 176},
  'scores': {'anger': 2.27258479e-05,
   'contempt': 0.00325852376,
   'disgust': 2.509685e-05,
   'fear': 6.535095e-08,
   'happiness': 0.416455328,
   'neutral': 0.57995373,
   'sadness': 0.0002770979,
   'surprise': 7.463036e-06}}]

In [116]:
#Turning Json to pandas dataframe

#function to flatten json objects
def flattenjson( b, delim ):
    val = {}
    for i in b.keys():
        if isinstance( b[i], dict ):
            get = flattenjson( b[i], delim )
            for j in get.keys():
                val[ i + delim + j ] = get[j]
        else:
            val[i] = b[i]

    return val

In [98]:
#select keys that are needed for analysis - it can be modified as needed
wanted_keys = ['faceAttributes', 'faceId', 'scores']

In [99]:
type(final_data)

list

In [117]:
f = []
for index, entry in enumerate(final_data):
    flat_dict = flattenjson(final_data[index], ",")
    trimmed_dict = dict((k, flat_dict[k]) for k in wanted_keys if k in flat_dict)
    f.append(trimmed_dict)

In [115]:
pd.DataFrame(final_data)

,faceAttributes,faceId,faceRectangle,scores
0,{'gender': 'male'},a5d0a88a-34ed-4367-88a9-9be6a4d0db3f,"{'left': 231, 'top': 108, 'height': 140, 'widt...","{'sadness': 0.579981, 'surprise': 0.0006214003..."
1,{'gender': 'female'},53ab45af-b792-45b6-8af2-4ef82c272b7b,"{'left': 628, 'top': 109, 'height': 136, 'widt...","{'sadness': 0.00117054419, 'surprise': 0.03964..."
